In [1]:
import torch
from brt.common import log
import brt
import brt.nn as nn
from brt.router import TopKScatterRouter

log.set_level("router", "DEBUG")
log.set_level("primitive", "DEBUG")


@brt.netlet
class MoE(nn.Module):
    def __init__(self):
        super().__init__()
        self.scatter_router = TopKScatterRouter(route_num=2, gran_dim=10)
        self.expert1 = nn.Linear(10, 10)
        self.expert2 = nn.Linear(10, 10)

    def forward(self, x):
        route_results, reverse_indice, reverse_shape = self.scatter_router(x)
        x_0 = self.expert1(route_results[0])
        x_1 = self.expert2(route_results[1])
        x = x_0 + x_1
        return x


@brt.domain
class MoEModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.moe = MoE()

    def forward(self, x):
        return self.moe(x)


moe_model = MoEModel()

x = torch.ones((60, 10))
x = x.cuda()
moe_model.cuda()
z = moe_model(x)
print(z)


using logger for brainstorm.primitive
using logger for brainstorm.router
setting logger for brainstorm.router to DEBUG level
setting logger for brainstorm.primitive to DEBUG level
<Logger brainstorm.router (DEBUG)>


dispatcher update time: 0.0005271434783935547


tensor([-1.7280, -0.3930,  0.2153, -0.2337, -0.7869,  1.0724, -0.8080,  0.1821,
         0.1815, -0.8225], device='cuda:0', grad_fn=<AddBackward0>)
